<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Cleaning-up-&amp;-recoding-hotgym.py-so-i-can-figure-out-how-it-works" data-toc-modified-id="Cleaning-up-&amp;-recoding-hotgym.py-so-i-can-figure-out-how-it-works-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cleaning up &amp; recoding hotgym.py so i can figure out how it works</a></span><ul class="toc-item"><li><span><a href="#default-parameter-dict" data-toc-modified-id="default-parameter-dict-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>default parameter dict</a></span><ul class="toc-item"><li><span><a href="#i'm-gonna-try-to-recode-this-without-&quot;def-main()&quot;-etc" data-toc-modified-id="i'm-gonna-try-to-recode-this-without-&quot;def-main()&quot;-etc-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>i'm gonna try to recode this without "def main()" etc</a></span></li><li><span><a href="#read-input-file" data-toc-modified-id="read-input-file-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>read input file</a></span></li><li><span><a href="#create-Encoder!" data-toc-modified-id="create-Encoder!-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>create Encoder!</a></span></li><li><span><a href="#create-spatial-pooler" data-toc-modified-id="create-spatial-pooler-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>create spatial pooler</a></span></li></ul></li><li><span><a href="#TM-parameter-notes:" data-toc-modified-id="TM-parameter-notes:-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>TM parameter notes:</a></span><ul class="toc-item"><li><span><a href="#setup-likelihood-/-anomaly-thresholds" data-toc-modified-id="setup-likelihood-/-anomaly-thresholds-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>setup likelihood / anomaly thresholds</a></span></li><li><span><a href="#iterate-through-dataset,-recording-inputs-&amp;-outputs" data-toc-modified-id="iterate-through-dataset,-recording-inputs-&amp;-outputs-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>iterate through dataset, recording inputs &amp; outputs</a></span></li></ul></li></ul></li></ul></div>

# Cleaning up & recoding hotgym.py so i can figure out how it works

In [5]:
import csv
import datetime
import os
import numpy as np
import random
import math

from htm.bindings.sdr import SDR, Metrics
from htm.encoders.rdse import RDSE, RDSE_Parameters
from htm.encoders.date import DateEncoder
from htm.bindings.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood #FIXME use TM.anomaly instead, but it gives worse results than the py.AnomalyLikelihood now
from htm.bindings.algorithms import Predictor

In [8]:
cwd = os.getcwd()
input_file_path = cwd+'/gymdata.csv'
input_file_path
# _INPUT_FILE_PATH = os.join(cwd, 'gymdata.csv') # i moved this locally from the example folder

'/Users/mark/Documents/flatiron/projects/htm/htm_nlp_predictive/gymdata.csv'

## default parameter dict

In [11]:
parameters = {
  # there are 2 (3) encoders: "value" (RDSE) & "time" (DateTime weekend, timeOfDay)
 'enc': {
      "value" :
         {'resolution': 0.88, 'size': 700, 'sparsity': 0.02},
      "time": 
         {'timeOfDay': (30, 1), 'weekend': 21}
 },
 'predictor': {'sdrc_alpha': 0.1},
 'sp': {'boostStrength': 3.0,
        'columnCount': 1638,
        'localAreaDensity': 0.04395604395604396,
        'potentialPct': 0.85,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'synPermInactiveDec': 0.006},
 'tm': {'activationThreshold': 17,
        'cellsPerColumn': 13,
        'initialPerm': 0.21,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'minThreshold': 10,
        'newSynapseCount': 32,
        'permanenceDec': 0.1,
        'permanenceInc': 0.1},
 'anomaly': {
   'likelihood': 
       {#'learningPeriod': int(math.floor(self.probationaryPeriod / 2.0)),
        #'probationaryPeriod': self.probationaryPeriod-default_parameters["anomaly"]["likelihood"]["learningPeriod"],
        'probationaryPct': 0.1,
        'reestimationPeriod': 100} #These settings are copied from NAB
 }
}

### i'm gonna try to recode this without "def main()" etc
- i must explore

In [13]:
import pprint
print("Parameters:")
pprint.pprint(parameters, indent=4)
# pprint('')

Parameters:
{   'anomaly': {   'likelihood': {   'probationaryPct': 0.1,
                                     'reestimationPeriod': 100}},
    'enc': {   'time': {'timeOfDay': (30, 1), 'weekend': 21},
               'value': {'resolution': 0.88, 'size': 700, 'sparsity': 0.02}},
    'predictor': {'sdrc_alpha': 0.1},
    'sp': {   'boostStrength': 3.0,
              'columnCount': 1638,
              'localAreaDensity': 0.04395604395604396,
              'potentialPct': 0.85,
              'synPermActiveInc': 0.04,
              'synPermConnected': 0.13999999999999999,
              'synPermInactiveDec': 0.006},
    'tm': {   'activationThreshold': 17,
              'cellsPerColumn': 13,
              'initialPerm': 0.21,
              'maxSegmentsPerCell': 128,
              'maxSynapsesPerSegment': 64,
              'minThreshold': 10,
              'newSynapseCount': 32,
              'permanenceDec': 0.1,
              'permanenceInc': 0.1}}


### read input file

In [14]:
records = []
with open(input_file_path, 'r') as fin:
    reader = csv.reader(fin)
    headers = next(reader)
    next(reader)
    next(reader)
    for record in reader:
        records.append(record) # gymdata.csv is just 2 columns
        # dateTime & float(power_consumption)


In [15]:
# records # list of dateTime, float
# they're coded as strings right now, but no worries

[['7/2/10 0:00', '21.2'],
 ['7/2/10 1:00', '16.4'],
 ['7/2/10 2:00', '4.7'],
 ['7/2/10 3:00', '4.7'],
 ['7/2/10 4:00', '4.6'],
 ['7/2/10 5:00', '23.5'],
 ['7/2/10 6:00', '47.5'],
 ['7/2/10 7:00', '45.4'],
 ['7/2/10 8:00', '46.1'],
 ['7/2/10 9:00', '41.5'],
 ['7/2/10 10:00', '43.4'],
 ['7/2/10 11:00', '43.8'],
 ['7/2/10 12:00', '37.8'],
 ['7/2/10 13:00', '36.6'],
 ['7/2/10 14:00', '35.7'],
 ['7/2/10 15:00', '38.9'],
 ['7/2/10 16:00', '36.2'],
 ['7/2/10 17:00', '36.6'],
 ['7/2/10 18:00', '37.2'],
 ['7/2/10 19:00', '38.2'],
 ['7/2/10 20:00', '14.1'],
 ['7/2/10 21:00', '5.1'],
 ['7/2/10 22:00', '5'],
 ['7/2/10 23:00', '5.9'],
 ['7/3/10 0:00', '22.5'],
 ['7/3/10 1:00', '22.3'],
 ['7/3/10 2:00', '6'],
 ['7/3/10 3:00', '4.7'],
 ['7/3/10 4:00', '4.4'],
 ['7/3/10 5:00', '4.8'],
 ['7/3/10 6:00', '23.4'],
 ['7/3/10 7:00', '42.4'],
 ['7/3/10 8:00', '52'],
 ['7/3/10 9:00', '50.8'],
 ['7/3/10 10:00', '45.3'],
 ['7/3/10 11:00', '43.6'],
 ['7/3/10 12:00', '40.7'],
 ['7/3/10 13:00', '38.4'],
 ['7/3/10 

### create Encoder!

In [18]:
parameters['enc']['time']['timeOfDay']

(30, 1)

In [20]:
parameters["enc"]["time"]["weekend"]

21

In [21]:
# ah, these parameters mean "how much SDR_space to allocate each variable"
dateEncoder = DateEncoder(
    timeOfDay = parameters['enc']['time']['timeOfDay'],
    weekend = parameters["enc"]["time"]["weekend"])

In [22]:
scalarEncoderParams = RDSE_Parameters() # random distributed scalar encoder
scalarEncoderParams.size       = parameters["enc"]["value"]["size"]
scalarEncoderParams.sparsity   = parameters["enc"]["value"]["sparsity"]
scalarEncoderParams.resolution = parameters["enc"]["value"]["resolution"]
scalarEncoder = RDSE(scalarEncoderParams)
encodingWidth = (dateEncoder.size + scalarEncoder.size)
enc_info = Metrics( [encodingWidth], 999999999) # wonder what this 2nd parameter is

In [24]:
scalarEncoderParams.resolution # ?

0.8799999952316284

### create spatial pooler

In [26]:
spParams = parameters['sp']

In [27]:
sp = SpatialPooler(
    inputDimensions            = (encodingWidth,),
    columnDimensions           = (spParams["columnCount"],),
    potentialPct               = spParams["potentialPct"],
    potentialRadius            = encodingWidth,
    globalInhibition           = True,
    localAreaDensity           = spParams["localAreaDensity"],
    synPermInactiveDec         = spParams["synPermInactiveDec"],
    synPermActiveInc           = spParams["synPermActiveInc"],
    synPermConnected           = spParams["synPermConnected"],
    boostStrength              = spParams["boostStrength"],
    wrapAround                 = True
)
sp_info = Metrics(sp.getColumnDimensions(), 999999999)

there's that "sp.getColumnDimensions()" again. very important method, i guess.

In [28]:
tmParams = parameters['tm']
tm = TemporalMemory(
    columnDimensions          = (spParams["columnCount"],),
    cellsPerColumn            = tmParams["cellsPerColumn"],
    activationThreshold       = tmParams["activationThreshold"],
    initialPermanence         = tmParams["initialPerm"],
    connectedPermanence       = spParams["synPermConnected"],
    minThreshold              = tmParams["minThreshold"],
    maxNewSynapseCount        = tmParams["newSynapseCount"],
    permanenceIncrement       = tmParams["permanenceInc"],
    permanenceDecrement       = tmParams["permanenceDec"],
    predictedSegmentDecrement = 0.0,
    maxSegmentsPerCell        = tmParams["maxSegmentsPerCell"],
    maxSynapsesPerSegment     = tmParams["maxSynapsesPerSegment"]
)
tm_info = Metrics( [tm.numberOfCells()], 999999999)
# we keep using the 999999999 thing whener we call Metrics(). hmmm

## TM parameter notes:
- cellsPerColumn is very interesting
    - if we only had 1, there would be no temporal aspect to the memory
    - each cell beyond 1 allows the TM to recognize state_A from another time-step before it
    - 4 cells can recognize len=3 state-chains, etc

### setup likelihood / anomaly thresholds

In [32]:
anParams = parameters['anomaly']['likelihood']
probationaryPeriod = int(math.floor(float(anParams['probationaryPct'])*len(records)))
learningPeriod = int(math.floor(probationaryPeriod / 2.0))
anomaly_history = AnomalyLikelihood(learningPeriod = learningPeriod,
       estimationSamples = probationaryPeriod - learningPeriod,
       reestimationPeriod = anParams['reestimationPeriod'])
predictor = Predictor(steps=[1,5], alpha=parameters['predictor']['sdrc_alpha'])
predictor_resolution = 1

### iterate through dataset, recording inputs & outputs

In [41]:
inputs = []
anomaly = []
anomalyProb = []
predictions = {1: [], 5:[]}
for count, record in enumerate(records): # iterate through listified CSV
    dateString = datetime.datetime.strptime(record[0], "%m/%d/%y %H:%M") # unstring it
    consumption = float(record[1]) # unstring the power value
    inputs.append(consumption) # add power to input
    
    # use encoder: create SDRs for each input value
    dateBits = dateEncoder.encode(dateString)
    consumptionBits = scalarEncoder.encode(consumption)
    
    # concatenate these encoded_SDRs into a larger one for pooling
    encoding = SDR(encodingWidth).concatenate([consumptionBits, dateBits])
    enc_info.addData(encoding) # enc_info is our metrics to keep track of how the encoder fares
    
    # create SDR to represent active columns. it'll be populated by .compute()
    # notably, this activeColumns SDR has same dimensions as spatial pooler
    activeColumns = SDR(sp.getColumnDimensions())
    
    # throw the input into the spatial pool and hope it swims
    sp.compute(encoding, True, activeColumns) # we're training, so learn=True
    tm_info.addData(tm.getActiveCells().flatten())
    
    # make prediction, then train the predictor accordingly
    pdf = predictor.infer( 
#         predictor, 
        tm.getActiveCells() 
    )
    for n in (1,5):
        if pdf[n]:
            predictions[n].append( np.argmax( pdf[n] ) * predictor_resolution )
        else:
            predictions[n].append(float('nan'))
    
    anomalyLikelihood = anomaly_history.anomalyProbability( consumption, tm.anomaly )
    anomaly.append(tm.anomaly)
    anomalyProb.append(anomalyLikelihood)
    
    predictor.learn(count, tm.getActiveCells(), int(consumption/predictor_resolution))


RuntimeError: CHECK FAILED: "dimensions_ != 0" Classifier: must call `learn` before `infer`.

## ah, we're still stuck at the same error as before
### something's written wrong, i think - or i've installed something wrong
- i can't find anything in the nupic docs regarding predictor.infer
- SDRClassifier.infer() needs a list of active indices from output + dict of classification label

In [35]:
type(predictor)

htm.bindings.algorithms.Predictor

In [ ]:
predictor.learn()

In [ ]:
predictor.infer()

In [36]:
inputs = []
anomaly = []
anomalyProb = []
predictions = {1: [], 5:[]}
for count, record in enumerate(records): # iterate through listified CSV
    dateString = datetime.datetime.strptime(record[0], "%m/%d/%y %H:%M") # unstring it
    consumption = float(record[1]) # unstring the power value
    inputs.append(consumption) # add power to input
    
    # use encoder: create SDRs for each input value
    dateBits = dateEncoder.encode(dateString)
    consumptionBits = scalarEncoder.encode(consumption)
    
    # concatenate these encoded_SDRs into a larger one for pooling
    encoding = SDR(encodingWidth).concatenate([consumptionBits, dateBits])
    enc_info.addData(encoding) # enc_info is our metrics to keep track of how the encoder fares
    
    # create SDR to represent active columns. it'll be populated by .compute()
    # notably, this activeColumns SDR has same dimensions as spatial pooler
    activeColumns = SDR(sp.getColumnDimensions())
    
    # throw the input into the spatial pool and hope it swims
    sp.compute(encoding, True, activeColumns) # we're training, so learn=True
    tm_info.addData(tm.getActiveCells().flatten())
    
    
    tm.compute(activeColumns, learn=True)
    
    # make prediction, then train the predictor accordingly
#     predictor.learn(activeColumns)
    pdf = predictor.infer( tm.getActiveCells() )
    for n in (1,5):
        if pdf[n]:
            predictions[n].append( np.argmax( pdf[n] ) * predictor_resolution )
        else:
            predictions[n].append(float('nan'))
    
    anomalyLikelihood = anomaly_history.anomalyProbability( consumption, tm.anomaly )
    anomaly.append(tm.anomaly)
    anomalyProb.append(anomalyLikelihood)
    
    predictor.learn(count, tm.getActiveCells(), int(consumption/predictor_resolution))

RuntimeError: CHECK FAILED: "dimensions_ != 0" Classifier: must call `learn` before `infer`.